In [ ]:
#!apt install sox libsndfile1 ffmpeg
#!pip3 install wheel
#!pip3 install -r requirements.txt
#!pip3 install wget unidecode
#!pip3 install resampy==0.3.1
#!pip3 install nemo_toolkit[all]
#!pip3 install numba==0.48
#!pip3 install librosa==0.8.1
#!pip3 install pynini

In [ ]:
import soundfile
import librosa
import json
import os

print(librosa.__version__)
def resample_audio(input_file_path, output_path, target_sampling_rate=22050):
    if not input_file_path.endswith(".wav"):
        raise NotImplementedError("Loading only implemented for wav files.")
    if not os.path.exists(input_file_path):
        raise FileNotFoundError(f"Cannot file input file at {input_file_path}")
    audio, sampling_rate = librosa.load(
      input_file_path,
      sr=target_sampling_rate
    )

    soundfile.write(
        output_path,
        audio,
        samplerate=target_sampling_rate,
        format="wav"
    )

In [ ]:
import requests
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
import shutil
import os
from bs4 import BeautifulSoup
import soundfile as sf
import string
import json
import re
import num2words
from tqdm.notebook import tqdm

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

blocklist = ["potato", "_ding_", "00_part1_entry-6", "_escape_"]
audio_dir = 'audio'
download_threads = 64

temp_path = "temp_audio"
sampling_rate = 22050
   
def prep(args, overwrite=True):
    already_exists = os.path.exists(audio_dir)
    
    if already_exists and not overwrite:
        print("Data already downloaded")
        return
    
    if already_exists:
        print("Deleting previously downloaded audio")
        shutil.rmtree(audio_dir)
        
        if os.path.exists(temp_path):
            shutil.rmtree(temp_path)
    
    os.mkdir(audio_dir)
    download_parallel(args)

def remove_punctuation(str):
    return str.translate(str.maketrans('', '', string.punctuation))
    
def audio_duration(fn):
    f = sf.SoundFile(fn)
    return f.frames / f.samplerate

def download_file(args):
    url, filename = args[0], args[1]

    try:
        response = requests.get(url, allow_redirects=False)

        open(os.path.join(audio_dir, filename), "wb").write(response.content)
        return filename, True
    except:
        return filename, False

def download_parallel(args):
    results = ThreadPool(download_threads).imap_unordered(download_file, args)
    for result in results:
        if result[1]:
            print(bcolors.OKGREEN + "[" + u'\u2713' + "] " + bcolors.ENDC + result[0])
        else:
            print(bcolors.FAIL + "[" + u'\u2715' + "] " + bcolors.ENDC + result[0])

def main():
    r = requests.get("https://theportalwiki.com/wiki/GLaDOS_voice_lines", allow_redirects=False)

    urls = []
    filenames = []
    texts = []

    soup = BeautifulSoup(r.text.encode('utf-8').decode('ascii', 'ignore'), 'html.parser')
    for link_item in soup.find_all('a'):
        url = link_item.get("href", None)
        if url:
            if "https:" in url and ".wav" in url:
                list_item = link_item.find_parent("li")
                ital_item = list_item.find_all('i')
                if ital_item:
                    text = ital_item[0].text
                    text = text.replace('"', '')
                    filename = url[url.rindex("/")+1:]

                    if "[" not in text and "]" not in text and "$" not in text:
                        if url not in urls:
                            for s in blocklist:
                                if s in url:
                                    break
                            else:
                                urls.append(url)
                                filenames.append(filename)
                                text = text.replace('*', '')
                                texts.append(text)

    print("Found " + str(len(urls)) + " urls")

    args = zip(urls, filenames)

    prep(args)
    
    total_audio_time = 0
    outFile=open(os.path.join(audio_dir, "manifest.json"), 'w')
    for i in range(len(urls)):
        item = {}
        text = texts[i]
        filename = filenames[i]
        item["audio_filepath"] = os.path.join(audio_dir, filename)
        #item["text_normalized"] = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)
        item["text"] = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text).lower()
        item["duration"] = audio_duration(os.path.join(audio_dir, filename))
        total_audio_time = total_audio_time + item["duration"]
        outFile.write(json.dumps(item, ensure_ascii=True, sort_keys=True) + "\n")
 
    outFile.close()
    print("\n" + str(total_audio_time/60.0) + " min\n")

main()

shutil.copytree(audio_dir, temp_path)

print("Resampling Audio...")
for filename in tqdm(os.listdir(temp_path)):
    if ".wav" in filename:
        source_name = os.path.join(temp_path, filename)
        destination_name = os.path.join(audio_dir, filename)
        resample_audio(source_name, destination_name, target_sampling_rate=sampling_rate)
        
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)

In [ ]:
!head -n 1 ./audio/manifest.json

In [ ]:
!cat ./audio/manifest.json | tail -n 5 > ./manifest_validation.json
!cat ./audio/manifest.json | head -n -5 > ./manifest_train.json

In [ ]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

In [ ]:
import os
import json

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

from nemo.collections.tts.models import FastPitchModel
FastPitchModel.from_pretrained("tts_en_fastpitch")

from pathlib import Path
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

In [ ]:
#!wget https://raw.githubusercontent.com/nvidia/NeMo/r1.14.0/examples/tts/fastpitch_finetune.py
#!wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.14.0/examples/tts/hifigan_finetune.py
#    
#!mkdir -p conf
#!cd conf \
#&& wget https://raw.githubusercontent.com/nvidia/NeMo/r1.14.0/examples/tts/conf/fastpitch_align_v1.05.yaml \
#&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.14.0/examples/tts/conf/hifigan/hifigan.yaml \
#&& cd ..

In [ ]:
# additional files
#!mkdir -p tts_dataset_files && cd tts_dataset_files \
#&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.14.0/scripts/tts_dataset_files/cmudict-0.7b_nv22.10 \
#&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.14.0/scripts/tts_dataset_files/heteronyms-052722 \
#&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.14.0/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
#&& cd ..

In [ ]:
!(python3 fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
    train_dataset=./manifest_train.json \
    validation_datasets=./manifest_validation.json \
    sup_data_path=./fastpitch_sup_data \
    phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.10 \
    heteronyms_path=tts_dataset_files/heteronyms-052722 \
    whitelist_path=tts_dataset_files/lj_speech.tsv \
    exp_manager.exp_dir=./glados_out \
    +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
    trainer.max_epochs=150 \
    trainer.check_val_every_n_epoch=25 \
    model.train_ds.dataloader_params.batch_size=12 model.validation_ds.dataloader_params.batch_size=12 \
    model.n_speakers=1 model.pitch_mean=165.458 model.pitch_std=40.1891 \
    model.pitch_fmin=80.0 model.pitch_fmax=2048.0 model.optim.lr=2e-4 \
    ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
    +model.text_tokenizer.add_blank_at=true \
)

In [ ]:
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

In [ ]:
import torch

def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir="./glados_out", 
        model_name="FastPitch"
    ):    
        
    exp_dirs = list([i for i in (Path(base_dir) / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [ ]:
import json
import IPython.display as ipd

# Only need to set speaker_id if there is more than one speaker
speaker_id = None

last_ckpt = get_best_ckpt_from_last_run(model_name="FastPitch")
print(last_ckpt)

spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

num_val = 5  # Number of validation samples
val_records = []
with open("manifest_validation.json", "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break
            
for val_record in val_records:
    print("Real validation audio")
    ipd.display(ipd.Audio(val_record['audio_filepath'], rate=22050))
    print(f"Text: {val_record['text']}")
    spec, audio = infer(spec_model, vocoder, val_record['text'], speaker=speaker_id)
    ipd.display(ipd.Audio(audio, rate=22050))

In [ ]:
import json
import numpy as np
import torch
import soundfile as sf

from pathlib import Path

from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator

def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()
    
def generateMels(manifest_path, save_path, hifigan_manifest_path):
    # Get records from the training manifest
  
    records = []
    with open(manifest_path, "r") as f:
        for i, line in enumerate(f):
            records.append(json.loads(line))

    beta_binomial_interpolator = BetaBinomialInterpolator()
    spec_model.eval()

    device = spec_model.device

    save_dir = Path(save_path)
    save_dir.mkdir(exist_ok=True, parents=True)

    # Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
    for i, r in enumerate(records):
        audio = load_wav(r["audio_filepath"])
        audio = torch.from_numpy(audio).unsqueeze(0).to(device)
        audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)

        # Again, our finetuned FastPitch model doesn't use multiple speakers,
        # but we keep the code to support it here for reference
        if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
            speaker = torch.tensor([r['speaker']]).to(device)
        else:
            speaker = None

        with torch.no_grad():
            if "normalized_text" in r:
                text = spec_model.parse(r["normalized_text"], normalize=False)
            else:
                text = spec_model.parse(r['text'])

            text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)

            spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

            # Generate attention prior and spectrogram inputs for HiFi-GAN
            attn_prior = torch.from_numpy(
              beta_binomial_interpolator(spect_len.item(), text_len.item())
            ).unsqueeze(0).to(text.device)

            spectrogram = spec_model.forward(
              text=text, 
              input_lens=text_len, 
              spec=spect, 
              mel_lens=spect_len, 
              attn_prior=attn_prior,
              speaker=speaker,
            )[0]

            save_path = save_dir / f"mel_{i}.npy"
            np.save(save_path, spectrogram[0].to('cpu').numpy())
            r["mel_filepath"] = str(save_path)

    
    with open(hifigan_manifest_path, "w") as f:
        for r in records:
            f.write(json.dumps(r) + '\n')

generateMels("./manifest_train.json", "./training_mels", "hifigan_manifest_train.json")
generateMels("./manifest_validation.json", "./validation_mels", "hifigan_manifest_validation.json")

In [ ]:
!(python3 hifigan_finetune.py \
    --config-name=hifigan.yaml \
    model.train_ds.dataloader_params.batch_size=32 \
    model.max_steps=7000 \
    model.optim.lr=0.00001 \
    ~model.optim.sched \
    train_dataset=./hifigan_manifest_train.json \
    validation_datasets=./hifigan_manifest_validation.json \
    exp_manager.exp_dir=./glados_out \
    +init_from_pretrained_model=tts_hifigan \
    trainer.check_val_every_n_epoch=10 \
    model/train_ds=train_ds_finetune \
    model/validation_ds=val_ds_finetune)

In [ ]:
import json
import IPython.display as ipd
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
import shutil
import os

# Only need to set speaker_id if there is more than one speaker
speaker_id = None

base_dir="glados_out"

last_fastpitch_ckpt = get_best_ckpt_from_last_run(base_dir=base_dir, model_name="FastPitch")
last_hifigan_ckpt = get_best_ckpt_from_last_run(base_dir=base_dir, model_name="HifiGan")

print("Fastpitch: " + last_fastpitch_ckpt)
print("HifiGan: " + last_hifigan_ckpt)

vocoder = HifiGanModel.load_from_checkpoint(last_hifigan_ckpt)
vocoder = vocoder.eval().cuda()
spec_model = FastPitchModel.load_from_checkpoint(last_fastpitch_ckpt)
spec_model.eval().cuda()

num_val = 5  # Number of validation samples
val_records = []
with open("manifest_validation.json", "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break
            
for val_record in val_records:
    print("Real validation audio")
    ipd.display(ipd.Audio(val_record['audio_filepath'], rate=22050))
    print(f"Text: {val_record['text']}")
    spec, audio = infer(spec_model, vocoder, val_record['text'], speaker=speaker_id)
    ipd.display(ipd.Audio(audio, rate=22050))

In [ ]:
spec, audio = infer(spec_model, vocoder, "Robots are awesome", speaker=speaker_id)
ipd.display(ipd.Audio(audio, rate=22050))
spec, audio = infer(spec_model, vocoder, "Welcome back to the armoury", speaker=speaker_id)
ipd.display(ipd.Audio(audio, rate=22050))

In [ ]:
spec_model.save_to("fastpitch.nemo")
vocoder.save_to("hifigan.nemo")

In [ ]:
# pip3 install whl
# pip3 install nemo2riva

# Run on training machine
!nemo2riva --out hifigan.riva hifigan.nemo --key None
!nemo2riva --out fastpitch.riva fastpitch.nemo --key None

# scp .riva files to jetson under /home/davesarmoury/RIVA/artifacts
# Everything below is run on the jetson

# Servicemaker docker
docker run --gpus all -it --rm \
    -v /home/davesarmoury/RIVA/artifacts:/servicemaker-dev \
    -v /home/davesarmoury/RIVA/riva_repo:/data \
    --entrypoint=\"/bin/bash\" \
     nvcr.io/nvidia/riva/riva-speech:2.13.1-servicemaker-l4t-aarch64

riva-build speech_synthesis \
    /servicemaker-dev/glados.rmir:tlt_encode \
    /servicemaker-dev/glados_fastpitch.riva:tlt_encode \
    /servicemaker-dev/glados_hifigan.riva:tlt_encode \
    --voice_name=GLaDOS \
    --sample_rate 22050

riva-deploy /servicemaker-dev/glados.rmir:tlt_encode /data/models

# Exit docker

ngc registry resource download-version nvidia/riva/riva_quickstart
cd riva_quickstart
bash riva_init.sh
# copy glados.riva files into riva models dir
bash riva_start.sh